In [26]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [39]:
# coding: utf-8

import os
import sys
from math import log
import numpy as np
incrementer = 1.0


def getFileContents(filename):
    data = None
    with open(filename, 'r') as f:
        data = f.readlines()
    return data


def getFileFromCommandLine():
    filename = sys.argv[1]
    return getFileContents(filename)



def splitWordTag(word_tag_pair):
    splitted = word_tag_pair.split('/')
    tag = splitted[-1]
    word = '/'.join(splitted[:-1])
    return word, tag



def getUniqueTags(tagged_data):
    tags = {}
    for line in tagged_data:
        word_tag_pairs = line.strip().split(' ')
        for word_tag_pair in word_tag_pairs:
            word, tag = splitWordTag(word_tag_pair)
            if tag in tags.keys():
                tags[tag] += 1
            else:
                tags[tag] = 1
    return tags


def getUniqueWords(tagged_data):
    words = []
    for line in tagged_data:
        word_tag_pairs = line.strip().split(' ')
        
        for word_tag_pair in word_tag_pairs:
            word, tag = splitWordTag(word_tag_pair)
            words.append(word)
    return list(set(words))


def readModelFile():
    filename = 'hmmmodel.txt'
    lines = []
    with open(filename, 'r') as model_file:
        lines = model_file.readlines()
    return lines


def parseModel(lines):
    total_tags = int(lines[0].strip().split(':')[-1])
    total_words = int(lines[1].strip().split(':')[-1])
    
    tr_start_line_number = int(lines[2].strip().split(':')[-2])
    tr_end_line_number = int(lines[2].strip().split(':')[-1])
    
    em_start_line_number = int(lines[3].strip().split(':')[-2])
    em_end_line_number = int(lines[3].strip().split(':')[-1])
    
    oc_start_line_number = int(lines[4].strip().split(':')[-2])
    oc_end_line_number = int(lines[4].strip().split(':')[-1])
    
    wi_start_line_number = int(lines[5].strip().split(':')[-2])
    wi_end_line_number = int(lines[5].strip().split(':')[-1])
    
    print total_tags, total_words, tr_start_line_number, tr_end_line_number, em_start_line_number, em_end_line_number, oc_start_line_number,oc_end_line_number, wi_start_line_number, wi_end_line_number
    
    probability_transition_matrix = []
    for line_number in range(tr_start_line_number, tr_end_line_number, 1):
        row_values = map(float, lines[line_number].strip().split('\t'))
        probability_transition_matrix.append(row_values)
    
    probability_emission_matrix = []
    for line_number in range(em_start_line_number, em_end_line_number, 1):
        row_values = map(float, lines[line_number].strip().split('\t'))
        probability_emission_matrix.append(row_values)
        
    
    opening_probabilities = {}
    closing_probabilities = {}
    
    tags_index_dict = {}
    tags_index_dict_reverse = {}
    
    for line_number in range(oc_start_line_number, oc_end_line_number, 1):
        row_values = lines[line_number].strip().split('\t')
        tag_name = row_values[0]
        open_p = float(row_values[1])
        close_p = float(row_values[2])
        index = int(row_values[3])
        
        opening_probabilities[tag_name] = open_p
        closing_probabilities[tag_name] = close_p
        tags_index_dict[tag_name] = index
        tags_index_dict_reverse[index] = tag_name
    
    words_index_dict = {}
    words_index_dict_reverse = {}
    
    for line_number in range(wi_start_line_number, wi_end_line_number, 1):
        row_values = lines[line_number].strip().split('\t')
        word = row_values[0]
        index = int(row_values[1])
        words_index_dict[word] = index
        words_index_dict_reverse[index] = word
        
    return opening_probabilities, closing_probabilities, probability_transition_matrix, probability_emission_matrix, tags_index_dict, tags_index_dict_reverse, words_index_dict, words_index_dict_reverse 



def getMostProbableTags(sentence):
    global opening_probabilities, closing_probabilities, probability_transition_matrix, probability_emission_matrix, tags_index_dict, tags_index_dict_reverse, words_index_dict, words_index_dict_reverse 
    global tag_count, unseen_words
    
    sentence_words = sentence.strip().split(' ')
    
    sentence_len = len(sentence_words)
    
    viterbi_matrix = np.zeros(shape=(tag_count, sentence_len))
    
    tracing_matrix = [[None for x in range(sentence_len)] for y in range(tag_count)]
    
    for col in range(sentence_len):
        word = sentence_words[col]
        for model_tag in tags_index_dict:
            model_tag_index = tags_index_dict[model_tag]
            try:
                word_emission_probability = probability_emission_matrix[model_tag_index][words_index_dict[word]]
                if word_emission_probability == 0.0:
                    continue
            except KeyError as e:
#                 print "probability_emission_matrix : Keyerror encountered"
                word_emission_probability = 1.0 #probability_emission_matrix[model_tag_index][-1]
            
            if col == 0:
                try:
                    tag_opening_probability = opening_probabilities[model_tag]
                except KeyError as e:
                    print "tag_opening_probability : Keyerror encountered"
                    tag_opening_probability = 1.1754943508222875e-100
                viterbi_matrix[model_tag_index][col] = tag_opening_probability * word_emission_probability
            else:
                max_probability = 0.0 #np.finfo(float).min
                max_tag = None
                
                for prev_model_tag in tags_index_dict:
                    prev_model_tag_index = tags_index_dict[prev_model_tag]
                    tag_transition_probability = probability_transition_matrix[prev_model_tag_index][model_tag_index]
                    if tag_transition_probability == 0.0:
                        print "Transition probability still zero"
                        tag_transition_probability = 1.1754943508222875e-100
                    temp_probability = viterbi_matrix[prev_model_tag_index][col-1] * tag_transition_probability * word_emission_probability  
                    if temp_probability > max_probability:
                        max_probability = temp_probability
                        max_tag = prev_model_tag
                        
                viterbi_matrix[model_tag_index][col] = max_probability
                tracing_matrix[model_tag_index][col] = max_tag
    
    max_probability = 0.0 #np.finfo(float).min
    max_probability_tag = None
    for model_tag in tags_index_dict:
        model_tag_index = tags_index_dict[model_tag]
        temp_probability = 0.0
        try:
            tag_closing_probabilities = closing_probabilities[model_tag]
        except KeyError as e:
            print "tag_closing_probabilities : Keyerror encountered", 
            tag_closing_probabilities = 1.1754943508222875e-100
        temp_probability =  tag_closing_probabilities * viterbi_matrix[model_tag_index][sentence_len-1]
        if temp_probability > max_probability:
            max_probability = temp_probability
            max_probability_tag = model_tag

    assigned_tags = [max_probability_tag]
    current_best_tag = max_probability_tag
    
    for col in range(sentence_len-1, 0, -1):
        current_best_tag = tracing_matrix[tags_index_dict[current_best_tag]][col]
        assigned_tags.append(current_best_tag)
    assigned_tags = assigned_tags[::-1]
    
    anotated_sentence = ''
    for index, assigned_tag in enumerate(assigned_tags):
        anotated_sentence += str(sentence_words[index]) + '/' + str(assigned_tag) + ' '
    
    
    return anotated_sentence.strip()

def startPredicting():
#     test_data = getFileFromCommandLine()
    test_data = getFileContents('data/zh_dev_raw.txt')
    output = ''
    for test_line in test_data:
        predicted_tagged_line = getMostProbableTags(test_line)
        output += predicted_tagged_line + '\n'
    
    with open('hmmoutput.txt', 'w') as output_file:
        output_file.write(output)



if __name__ == '__main__':
    import time
    start = time.time()
    lines = readModelFile()
    opening_probabilities, closing_probabilities, probability_transition_matrix, probability_emission_matrix, tags_index_dict, tags_index_dict_reverse, words_index_dict, words_index_dict_reverse  = parseModel(lines)
    tag_count = len(tags_index_dict.keys())
    startPredicting()
    print "Done"
    print time.time() - start

42 17610 7 49 50 92 93 135 136 17746
Done
4.63826107979


In [36]:
def getFileContents(filename):
    data = None
    with open(filename, 'r') as f:
        data = f.readlines()
    return data

def computeAccuracy():
    dev_tagged_data = getFileContents('data/zh_dev_tagged.txt')
    predicted_data = getFileContents('hmmoutput.txt')
    correct = 0
    total = 0
    for index, line in enumerate(dev_tagged_data):
        predicted_tagged_line = predicted_data[index]
        expected_tagged_line = dev_tagged_data[index]
        
        predicted_word_tag_pairs = predicted_tagged_line.strip().split(' ')
        expected_word_tag_pairs = expected_tagged_line.strip().split(' ')
        for index, predicted_word in enumerate(predicted_word_tag_pairs):
            if predicted_word == expected_word_tag_pairs[index]:
                correct += 1
            total += 1
            if total % 100 == 0:
                print correct, total, " => ", (correct*100.0)/total
    accuracy = (correct*100.0)/total
    print accuracy

computeAccuracy()

86 100  =>  86.0
171 200  =>  85.5
253 300  =>  84.3333333333
337 400  =>  84.25
421 500  =>  84.2
514 600  =>  85.6666666667
600 700  =>  85.7142857143
686 800  =>  85.75
772 900  =>  85.7777777778
857 1000  =>  85.7
945 1100  =>  85.9090909091
1040 1200  =>  86.6666666667
1127 1300  =>  86.6923076923
1213 1400  =>  86.6428571429
1295 1500  =>  86.3333333333
1379 1600  =>  86.1875
1464 1700  =>  86.1176470588
1544 1800  =>  85.7777777778
1621 1900  =>  85.3157894737
1704 2000  =>  85.2
1783 2100  =>  84.9047619048
1866 2200  =>  84.8181818182
1953 2300  =>  84.9130434783
2046 2400  =>  85.25
2129 2500  =>  85.16
2213 2600  =>  85.1153846154
2302 2700  =>  85.2592592593
2392 2800  =>  85.4285714286
2481 2900  =>  85.5517241379
2563 3000  =>  85.4333333333
2653 3100  =>  85.5806451613
2744 3200  =>  85.75
2840 3300  =>  86.0606060606
2929 3400  =>  86.1470588235
3020 3500  =>  86.2857142857
3102 3600  =>  86.1666666667
3185 3700  =>  86.0810810811
3269 3800  =>  86.0263157895
3353 3900 